In [1]:

from toolsdsgen import *

In [2]:
cam_poses = set_cam_poses()
t_r2w, t_w2r = get_transmats(cam_poses)
r_r2w, r_w2r = get_rotmats(cam_poses)
focl = set_focal_lengths()

# Object pose in world space
μ_w = np.array([-0.25, 1.00, 0.50])

tete = np.zeros((3,3))
titi = np.zeros((4,3))

# Object pose in real-camera spaces (x3)
μ_r = np.zeros((3,3)) # including a '1' at the end
μ_f = np.zeros((4,4)) # 

cam_ang_fp = np.zeros((2,3))
px = np.zeros(3)
py = np.zeros(3)

# Σ = np.array([[0.02,0,0],[0,0.02,0],[0,0,1.8]])    


for k in range(3):

    
    titi[:,k] = t_w2r[:,:, k] @ np.concatenate((μ_w,[1]))
    μ_r[:,k]  = r_w2r[:,:,k] @ titi[0:3,k]
    
    # Going Back to World Space from Real-Camera Space
    tete[:,k] = r_r2w[:,:,k] @ μ_r[0:3,k]
    μ_f[:,k] = t_r2w[:,:, k] @ np.concatenate((tete[:,k],[1]))
    
    # Estimating Which pixels are concerned
    cam_ang_fp[:,k] = get_angles_from_pos(μ_r[:,k])
    px[k], py[k] = get_dvs_from_angles(cam_ang_fp[:,k], focl, k)
    
    

print("Poses in Real-Camera Space")
print(μ_r[:,0])
print(μ_r[:,1])
print(μ_r[:,2])

print("Coming Back to World Space")
print(μ_f[0:3,0])
print(μ_f[0:3,1])
print(μ_f[0:3,2])

print("Angles")
print(cam_ang_fp[:,0])
print(cam_ang_fp[:,1])
print(cam_ang_fp[:,2])


print("Pixels")

print(f"Cam 1: x:{px[0]} y:{py[0]}")
print(f"Cam 2: x:{px[1]} y:{py[1]}")
print(f"Cam 3: x:{px[2]} y:{py[2]}")


Poses in Real-Camera Space
[ 0.1767767   0.         -0.53033009]
[-0.1767767   0.         -0.53033009]
[ 1.5308085e-17  0.0000000e+00 -2.5000000e-01]
Coming Back to World Space
[-0.25  1.    0.5 ]
[-0.25  1.    0.5 ]
[-0.25  1.    0.5 ]
Angles
[18.43494882  0.        ]
[-18.43494882   0.        ]
[0. 0.]
Pixels
Cam 1: x:216.4099493333335 y:0.0
Cam 2: x:-237.6634999999998 y:0.0
Cam 3: x:0.0 y:0.0


In [3]:
# THIS IS JUST TO DOUBLE CHECK PROCESS!
cam_ang_fd = np.zeros((2,3))
for k in range(3):
    
    cam_ang_fd[:,k] = get_angles_from_dvs(px[k], py[k], focl, k)

v_poses = set_vir_poses(cam_ang_fd)

r_v2r, r_r2v = get_rotmats(v_poses)

μ = np.array([0,0,-0.95])
Σ = np.array([[0.05,0,0],[0,0.05,0],[0,0,20]])    

new_μ, w_Σ, v_Σ = create_mgd(μ, Σ, r_v2r, r_r2w, t_r2w)

In [5]:
μ_p = analytical(new_μ, w_Σ)
μ_d = abs(np.array([μ_p[0]-μ_w[0], μ_p[1]-μ_w[1], μ_p[2]-μ_w[2]])*1000)
d = math.sqrt((μ_p[0]-μ_w[0])**2+(μ_p[1]-μ_w[1])**2+(μ_p[2]-μ_w[2])**2)

print(f"μ_w: {μ_w} [m]")
print(f"μ_p: {μ_p} [m]")
print(f"μ_d: {μ_d} [mm]")
print(f"delta: {round(d*1000,1)}[mm]")

print("I m checking delta depending on SIGMA")

μ_w: [-0.25  1.    0.5 ] [m]
μ_p: [-0.24890864  1.          0.49875461] [m]
μ_d: [1.09136264 0.         1.24539114] [mm]
delta: 1.7[mm]
I m checking delta depending on SIGMA
